In [354]:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import date
from datetime import datetime
from sklearn.model_selection import train_test_split

In [355]:
# setting saving directory
today = date.today()
now = datetime.now()
print (today)
print (now)
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S") #do not put : or similar 
print("date and time =", dt_string)	    
root = "C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs"

#saving function
def saveAsCSV(df,name,rootdir = root):
    """
    df = dataframe
    name = name for saving in format: 'thisisthename.csv'
    rootdir = local saving directory, default is root
    """
    savepath = rootdir +'/' + dt_string+'_'+ name 
    df.to_csv ( savepath, index = False, header=True)

#loading function
def loadCSV(name,rootdir = root):
    """
    rootdir = local saving directory, default is root
    name = exact name (including datetime and extention .csv)
    """
    dt_string = now.strftime("%d-%m-%Y_%H-%M-%S") 
    loadpath = rootdir +'/'+dt_string+'_' +name 
    print ("loading: ",loadpath)
    df = pd.read_csv(loadpath)
    return df
    

2020-06-21
2020-06-21 11:20:39.754150
date and time = 21-06-2020_11-20-39


In [356]:
#here all the functions (for when we will use many df)

def keep1(arrayorint):
    """
    array = array of numbers
    """
    if (type(arrayorint) != int):
        new_list =[] 
        for num in arrayorint:
            first_digit = int(str(num)[:1]) 
            new_list.append(first_digit)
        new = np.array(new_list)
    else: 
        new = int(str(arrayorint)[:1]) 
    return new

def keepOne(arrayorint):
    """
    array = array of numbers
    """
    if (type(arrayorint) != int):
        new_list =[] 
        for num in arrayorint:
            first_digit = int(np.floor(num / 1000))
            new_list.append(first_digit)
        new = np.array(new_list)
    else: 
        new = int(np.floor(arrayorint / 1000))
    return new


def infoDataFrame(dt_test):
    #how many entries in this dataset? 
    tot_rows= len(dt_test)
    print ("tot num entries in dataset: ",tot_rows)

    #how many hh in this dataset? 
    tot_hh  = len(dt_test.serial.unique())
    print ("tot num hh in dataset: ",tot_hh)

    #how many people in this dataset? 
    tot_ppl = len(dt_test.person_id.unique())
    print ("tot num people in dataset: ",tot_ppl )

    #how many rows in training and how many in testing? 
    train = np.floor(tot_rows*0.8)
    print ("train: %i,   test: %i" %(train,tot_rows-train))

def addUniquePersonID(dt_test):
    dt_test["person_id"] = dt_test["serial"] + dt_test["pnum"]/100
    return dt_test


def addNextActivity(df_test):
    whatisnext = []
    for i in range(len(df_test)):

        # Get details of row
        current_person_id = df_test['person_id'][i]
        current_activity = df_test['whatdoing_exact'][i]

        if i < (len(df_test)-1):
            #check next row
            next_person_id = df_test['person_id'][i+1]
            next_activity = df_test['whatdoing_exact'][i+1]

            # check and add
            if (current_person_id == next_person_id):
                whatisnext.append(next_activity)

            else: #assumption: we assume the activity is continued
                whatisnext.append(current_activity)

        else: #assumption: we assume the activity is continued
            #print (i)
            whatisnext.append(current_activity)

    whatisnext = pd.Series(whatisnext, name = 'Next_activity')
    new_column_df = whatisnext.to_frame()    

    #we check for missing values, if the new column is shorter--> erron in the concatenation!
    if (len(df_test) != len(whatisnext)):
        print("original len: %i, modified len: %i"  %(len(df_test) ,len(whatisnext)))

    #concat on axis = 1 --> we add one column
    df_test = pd.concat([df_test, new_column_df],axis=1)
    return df_test

def createTransitionMatrix (df_test):
    #get the indexes 
    all_indexes = df_test.index.values.tolist()

    #sizing 
    all_next_act= df_test['Next_activity']
    simplified_all_next_act = keepOne(all_next_act)
    unique_next_act =np.unique(simplified_all_next_act)
    #print (unique_next_act)
    n = len(unique_next_act)
    print ("Number of unique Activities:",n)

    # Init
    total_tp = 144
    P_t = np.zeros([total_tp, n, n]) # <- 3D array for 144 * transition matrices

    # For debugging
    test = False

    # Loop through all activity entries again
    for i in all_indexes:
        #here we get double codes
        t = df_test['time_period'][i] # <- % 144 to take all days in the years
        j = df_test['whatdoing_exact'][i] # <- current state
        k = df_test['Next_activity'][i]  # <- next state
        #print (t,j,k)
        
        #here we simplify them into one (eg "31" ->"3", "32"->"3", "99"->9)
        j_simply = keepOne(j.item())
        k_simply = keepOne(k.item())
        #print (type(t),type(j_simply),type(k_simply))

        #we take their positional indexes from unique_next_act
        j_index= np.where(unique_next_act == j_simply) 
        k_index= np.where(unique_next_act == k_simply) 

        #to increase the right bin
        P_t[t, j_index, k_index] += 1 # <- add one to probability, we'll divide later
        
        #if test:
            #print(activ_array[i, :])
            #print("At time %0.0f, the person is transitioning from %0.0f to %0.0f" % (t, j, k))
            #print("------")
            
    for t in range(total_tp):
        P_t[t] = (P_t[t].T/np.sum(P_t[t], axis=1)).T # <- transpose and transpose back to match axes with division operator
        for row in range(n):
            if np.isnan(P_t[t, row]).any(): # <- replace nans with 0s
                P_t[t, row] = np.zeros(n)
       
    return P_t
    

In [357]:
# Load data
#df_senior = loadCSV('20-06-2020_21-18-24_df_senior.csv')

df_child = loadCSV('21-06-2020_11-14-42_df_child.csv')
df_adult1 = loadCSV('21-06-2020_11-14-42_df_adult1.csv')
df_adult2 = loadCSV('21-06-2020_11-14-42_df_adult2.csv')
df_adult3 = loadCSV('21-06-2020_11-14-42_df_adult3.csv')
df_adult4 = loadCSV('21-06-2020_11-14-42_df_adult4.csv')
df_senior = loadCSV('21-06-2020_11-14-42_df_senior.csv')

loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/21-06-2020_11-14-42_df_child.csv
loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/21-06-2020_11-14-42_df_adult1.csv
loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/21-06-2020_11-14-42_df_adult2.csv
loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/21-06-2020_11-14-42_df_adult3.csv
loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/21-06-2020_11-14-42_df_adult4.csv
loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/21-06-2020_11-14-42_df_senior.csv


In [358]:
#append them all in one vector
all_dataframes = []
all_dataframes.append(df_child)
all_dataframes.append(df_adult1)
all_dataframes.append(df_adult2)
all_dataframes.append(df_adult3)
all_dataframes.append(df_adult4)
all_dataframes.append(df_senior)

In [359]:
#append all dataframes_labels in one vector
all_dataframes_labels = []
all_dataframes_labels.append('df_child')
all_dataframes_labels.append('df_adult1')
all_dataframes_labels.append('df_adult2')
all_dataframes_labels.append('df_adult3')
all_dataframes_labels.append('df_adult4')
all_dataframes_labels.append('df_senior')

In [360]:
for i in range(len(all_dataframes)):
    print ("shape of %s : %s"%(all_dataframes_labels[i], all_dataframes[i].shape))

shape of df_child : (323200, 15)
shape of df_adult1 : (344341, 15)
shape of df_adult2 : (501748, 15)
shape of df_adult3 : (557492, 15)
shape of df_adult4 : (460642, 15)
shape of df_senior : (185534, 15)


In [364]:
#peek in columns and setting a new order
df_senior.head()
cols = list(df_senior)
#print (cols)

newcols = ['old_time_index',
 'person_id',
 'serial',
 'pnum',
 'Date',
 'Season',
 'day_of_week',
 'New_Time',
 'time_period',
 'whatdoing_exact',
 'sex',
 'age',
 'Time',
 't_start',
 'eptime']

In [371]:
# sort df 
def sortColumns(df_test, newcols):
    df_test = df_test.ix[:,newcols]
    return df_test
    

df_child = df_child.ix[:,newcols]
df_adult1 = df_adult1.ix[:,newcols]
df_adult2 = df_adult2.ix[:,newcols]
df_adult3 = df_adult3.ix[:,newcols]
df_adult4 = df_adult4.ix[:,newcols]
df_senior = df_senior.ix[:,newcols]

df_senior.head()

C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\arita\AppData\Loca

,old_time_index,person_id,serial,pnum,Date,Season,day_of_week,New_Time,time_period,whatdoing_exact,sex,age,Time,t_start,eptime
0,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:00:00,18,110,1,75,03:00:00,2014-12-07 03:00:00,10.0
1,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:10:00,19,110,1,75,03:00:00,2014-12-07 03:00:00,10.0
2,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:20:00,20,110,1,75,03:00:00,2014-12-07 03:00:00,10.0
3,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:30:00,21,110,1,75,03:00:00,2014-12-07 03:00:00,10.0
4,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:40:00,22,110,1,75,03:00:00,2014-12-07 03:00:00,10.0


In [372]:
# drop columns that are not useful
#df_senior = df_senior.drop(columns=['Time','eptime'])

df_child = df_child.drop(columns=['Time','eptime'])
df_adult1 = df_adult1.drop(columns=['Time','eptime'])
df_adult2 = df_adult2.drop(columns=['Time','eptime'])
df_adult3 = df_adult3.drop(columns=['Time','eptime'])
df_adult4 = df_adult4.drop(columns=['Time','eptime'])
df_senior = df_senior.drop(columns=['Time','eptime'])

df_senior.head()

,old_time_index,person_id,serial,pnum,Date,Season,day_of_week,New_Time,time_period,whatdoing_exact,sex,age,t_start
0,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:00:00,18,110,1,75,2014-12-07 03:00:00
1,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:10:00,19,110,1,75,2014-12-07 03:00:00
2,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:20:00,20,110,1,75,2014-12-07 03:00:00
3,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:30:00,21,110,1,75,2014-12-07 03:00:00
4,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:40:00,22,110,1,75,2014-12-07 03:00:00


In [374]:
for i in range(len(all_dataframes)):
    df_test = all_dataframes[i]
    print ("")
    print ("Check on %s"%(all_dataframes_labels[i]))
    infoDataFrame(df_test)


Check on df_child
tot num entries in dataset:  323200
tot num hh in dataset:  761
tot num people in dataset:  1128
train: 258560,   test: 64640

Check on df_adult1
tot num entries in dataset:  344341
tot num hh in dataset:  886
tot num people in dataset:  1203
train: 275472,   test: 68869

Check on df_adult2
tot num entries in dataset:  501748
tot num hh in dataset:  1277
tot num people in dataset:  1749
train: 401398,   test: 100350

Check on df_adult3
tot num entries in dataset:  557492
tot num hh in dataset:  1426
tot num people in dataset:  1941
train: 445993,   test: 111499

Check on df_adult4
tot num entries in dataset:  460642
tot num hh in dataset:  1189
tot num people in dataset:  1605
train: 368513,   test: 92129

Check on df_senior
tot num entries in dataset:  185534
tot num hh in dataset:  540
tot num people in dataset:  646
train: 148427,   test: 37107


In [375]:
#BEFORE splitting the dataset in training and test we should create a column with NEXTwhatisdoing 
#for each household, for each person
#giving that the lines are ordered in theory checking the pnum would be enough, but anyway we have the person_id!
#df_senior = addNextActivity(df_senior)

df_child = addNextActivity(df_child)
df_adult1 = addNextActivity(df_adult1)
df_adult2 = addNextActivity(df_adult2)
df_adult3 = addNextActivity(df_adult3)
df_adult4 = addNextActivity(df_adult4)
df_senior = addNextActivity(df_senior)

df_senior.head()

,old_time_index,person_id,serial,pnum,Date,Season,day_of_week,New_Time,time_period,whatdoing_exact,sex,age,t_start,Next_activity
0,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:00:00,18,110,1,75,2014-12-07 03:00:00,110
1,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:10:00,19,110,1,75,2014-12-07 03:00:00,110
2,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:20:00,20,110,1,75,2014-12-07 03:00:00,110
3,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:30:00,21,110,1,75,2014-12-07 03:00:00,110
4,142,11011203.01,11011203,1,2014-12-07,winter,Sunday,03:40:00,22,110,1,75,2014-12-07 03:00:00,110


In [376]:
#save matrix 
saveAsCSV(df_child,'df_senior_withnextactivity')
saveAsCSV(df_adult1,'df_senior_withnextactivity')
saveAsCSV(df_adult2,'df_senior_withnextactivity')
saveAsCSV(df_adult3,'df_senior_withnextactivity')
saveAsCSV(df_adult4,'df_senior_withnextactivity')
saveAsCSV(df_senior,'df_senior_withnextactivity')

In [377]:
#from sklearn.model_selection import train_test_split

# split datasets in train and test
testSize = 0.2
train_df_child, test_df_child = train_test_split(df_child, test_size=testSize)
train_df_adult1, test_df_adult1 = train_test_split(df_adult1, test_size=testSize)
train_df_adult2, test_df_adult2 = train_test_split(df_adult2, test_size=testSize)
train_df_adult3, test_df_adult3 = train_test_split(df_adult3, test_size=testSize)
train_df_adult4, test_df_adult4 = train_test_split(df_adult4, test_size=testSize)
train_df_senior, test_df_senior = train_test_split(df_senior, test_size=testSize)



In [378]:
#save them 
saveAsCSV(train_df_child,'train_df_child')
saveAsCSV(test_df_child,'test_df_child')

saveAsCSV(train_df_adult1,'train_df_adult1')
saveAsCSV(test_df_adult1,'test_df_adult1')

saveAsCSV(train_df_adult2,'train_df_adult2')
saveAsCSV(test_df_adult2,'test_df_adult2')

saveAsCSV(train_df_adult3,'train_df_adult3')
saveAsCSV(test_df_adult3,'test_df_adult3')

saveAsCSV(train_df_adult4,'train_df_adult4')
saveAsCSV(test_df_adult4,'test_df_adult4')

saveAsCSV(train_df_senior,'train_df_senior')
saveAsCSV(test_df_senior,'test_df_senior')

In [379]:
len(train_df_senior)
train_df_senior.head(25)
#perfect!

,old_time_index,person_id,serial,pnum,Date,Season,day_of_week,New_Time,time_period,whatdoing_exact,sex,age,t_start,Next_activity
31857,90355,12160205.01,12160205,1,2015-02-28,winter,Saturday,23:50:00,143,110,2,76,2015-02-28 22:10:00,110
20329,63675,12020319.01,12020319,1,2015-03-18,spring,Wednesday,16:10:00,97,5120,2,78,2015-03-18 15:50:00,210
55927,186950,14131007.01,14131007,1,2014-10-20,autumn,Monday,21:40:00,130,110,2,77,2014-10-20 21:10:00,110
27564,81231,12090618.02,12090618,2,2014-06-29,summer,Sunday,23:00:00,138,110,2,87,2014-06-29 23:00:00,110
50366,162157,13280702.01,13280702,1,2014-11-23,autumn,Sunday,03:00:00,18,110,1,90,2014-11-23 03:00:00,110
103293,339050,17050511.01,17050511,1,2014-05-25,spring,Sunday,21:40:00,130,110,1,75,2014-05-25 21:00:00,110
32384,92415,12160307.01,12160307,1,2015-04-04,spring,Saturday,14:40:00,88,8210,1,76,2015-04-04 13:00:00,8210
166439,513944,20220915.01,20220915,1,2015-01-14,winter,Wednesday,10:40:00,64,3210,2,79,2015-01-14 10:30:00,3210
22414,67915,12050504.01,12050504,1,2014-05-11,spring,Sunday,02:40:00,16,110,2,80,2014-05-11 02:00:00,110
51167,168934,14020301.01,14020301,1,2015-03-11,spring,Wednesday,16:30:00,99,5140,1,78,2015-03-11 16:30:00,3110


In [307]:
#all_indexes = df_test.index.values.tolist()
#print (np.unique (all_indexes))
#train_df_adult1['time_period'][0]

112

In [380]:
#create the transition matrix for each train df

P_t_child = createTransitionMatrix(train_df_child)
P_t_adult1 = createTransitionMatrix(train_df_adult1)
P_t_adult2 = createTransitionMatrix(train_df_adult2)
P_t_adult3 = createTransitionMatrix(train_df_adult3)
P_t_adult4 = createTransitionMatrix(train_df_adult4)
P_t_senior = createTransitionMatrix(train_df_senior)

Number of unique Activities: 10


C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:134: RuntimeWarning: invalid value encountered in true_divide


Number of unique Activities: 10
Number of unique Activities: 10
Number of unique Activities: 10
Number of unique Activities: 10
Number of unique Activities: 10


In [350]:
# looks good!
print ((P_t_senior))

[[[1.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 1.         0.         0.        ]
  [0.2        0.         0.         ... 0.         0.8        0.        ]
  [0.         0.         0.         ... 0.         0.         1.        ]]

 [[0.99737533 0.         0.         ... 0.         0.00262467 0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 1.         0.         0.        ]
  [0.         0.         0.         ... 0.         1.         0.        ]
  [0.         0.         0.         ... 0.         0.         1.        ]]

 [[1.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.      

In [382]:
import os
os.getcwd()
os.chdir('C:\\Users\\arita\\Documents\\GitHub\\02443-SimSolarEnergy\\code\\tm')

In [383]:
import pickle
#lets pickle the np array

with open( dt_string+'_'+'P_t_child.pkl','wb') as f:
    pickle.dump(P_t_child, f)
    
with open( dt_string+'_'+'P_t_adult1.pkl','wb') as f:
    pickle.dump(P_t_adult1, f)

with open( dt_string+'_'+'P_t_adult2.pkl','wb') as f:
    pickle.dump(P_t_adult2, f)

with open( dt_string+'_'+'P_t_adult3.pkl','wb') as f:
    pickle.dump(P_t_adult3, f)

with open( dt_string+'_'+'P_t_adult4.pkl','wb') as f:
    pickle.dump(P_t_adult4, f)

with open( dt_string+'_'+'P_t_senior.pkl','wb') as f:
    pickle.dump(P_t_senior, f)
    

In [352]:
all_act = train_df_senior.whatdoing_exact
simplified_all_act = keepOne(all_act)
unique_act =np.unique(simplified_all_act)

In [353]:
unique_act

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])